# Using different numerical integration schemes to evaluate the function
## f(x) = exp(-2x)cos(10x)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Defining the function to integrate

In [ ]:
def fxn(x):
    a = -2.
    b = 10.
    # the function is f(x) = exp(-2x)cos(10x)
    return np.exp(a*x)*np.cos(b*x)

## Defining the integral analytically for future comparison

In [ ]:
#Solved the integral separately
#Since this is for a definite integral, assume integration constants are zero
def fxn_int(x):
    a = -1/52
    b = 5/52
    c = -2
    d = 10
    # the integrated function is -1/52 exp(-2x)cos(10x) + 5/52 exp(-2x)sin(10x)
    return (a*np.exp(c*x)*np.cos(d*x)) + (b*np.exp(c*x)*np.sin(d*x))

## Modules for the Trapezoid Method

### Defining the Trapezoid Method Core

In [ ]:
#this performs the scheme over a single integral
def trapezoid_core(f,x,h):   
    #f is the function being integrated
    #x is the left side of the interval
    #h is the width of the interval
    #area of a trapezoid is A = 0.5h(b1 + b2)
    return 0.5*h*(f(x+h)+f(x)) #this is the left side of the interval

### Defining the Trapezoid Method Wrapper Function

In [ ]:
#this performs the scheme of totalling up the intervals
def trapezoid_method(f,a,b,N):
    #f is the function to integrate
    #a is the lower limit of integration
    #b is the upper limit of integration
    #N is the number of function evaluations to use
    
    #define x values to perform trapezoid rule (an array)
    #recall the trapezoid method evals at x and x(i+1)
    x = np.linspace(a,b,N) #an array of size N going from a to b with linear spacing
    h = x[1]-x[0] #this is the width, array has equal separation so only need once
    
    #define the value of the integral (sum of intervals)
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    #the region (and therefore loop) with N locations, have (N-1) intervals
    #left hand side with stop at N-2 to not include rightmost element
    for i in range(0,len(x)-1,1): #will do a loop from 0 to (N-2)th element, step of 1
        Fint += trapezoid_core(f,x[i],h)
        
    #return the answer - sum of areas of all the trapezoids
    return Fint

## Modules for the Simpson's Method

### Defining the Simpson's Method Core

In [ ]:
#This integrates a single interval
#This is a piecewise parabolic and will be summing 2 intervals at a time
# h* (1/3 f(xi) + 4/3 f(xi+1) + 2/3 f(xi+2))
def simpson_core(f,x,h):
    #f is the function to be integrated
    #x is the ith x element
    #h is the width of the interval
    return h*(f(x) + 4*f(x+h) + f(x + 2*h))/3

### Defining the Simpson's Method Wrapper Function

In [ ]:
#This sums up the intervals
def simpsons_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    #note that number of chunks will be N-1
    #so if N is odd, then we don't need to adjust the last segment
        #since we eval 2 chunks at a time
    
    #define x values to perform simpsons rule (an array)
    x = np.linspace(a,b,N) #an array of size N going from a to b
    h = x[1]-x[0] #this is the width, array has equal separation so only need once
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the Simpson's method
    for i in range(0,len(x)-2,2): #will do a loop from 0 to (N-3)th element, step of 2
        Fint += simpson_core(f,x[i],h)
        
    #apply simpson's rule over the last interval
    #if N is even
    if ((N%2)==0): #N modulo 2 = 0 if even
        #need to split inverval in half and perform simpson method for that last interval
        Fint += simpson_core(f, x[-2], 0.5*h)
        
    #return the answer - sum of areas of all the parabolics
    return Fint

## Modules for the Romberg Integration

### Defining the Romberg Integration Core

In [ ]:
def romberg_core(f,a,b,i):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #i == level of refinement
    
    #we need the width difference b-a 
    h = b-a
    
    #interval between function evaluations at refinement level i
    #the increment between new fxn evaluations
    dh = h / (2.**(i))
    
    #we need the cofactor
    #weights the sum of the fxn evaluations
    K = h / (2.**(i+1))
    
    #value of the sum of the evaluations
    M = 0.0  
    for j in range(2**i):
        #at every level of refinement i, we add additional function evaluations  
        M += f(a+ 0.5*dh + j*dh) 
        
    #return the answer
    #sum multiplied by the correction factor
    return K*M

### Defining the Romberg Integration Wrapper Function

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000 #will not even get close to this
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    #this is the first approximation, 
    #really just trapezoidal rule over the whole domain [a,b]
    I[0] = 0.5*(b-a)*(f(a)-f(b))  
    
    #iterate by 1
    i += 1
    
    #call romberg core until the error is less than the tolerance
    #iterate until then
    while(delta>tol):
        
        #finding this romberg iteration
        #1/2previous approx + correction term (romberg_core)
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)  
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )  #remember to use [] for elements in array!
        
        print("i:",i, "I[i]:", I[i],"I[i-1]:", I[i-1], "delta:",delta)
        
        if(delta>tol):
            
            #iterate
            i+=1
            
            #if we've reached the maximum iterations/intervals
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
                
    #printing number of iterations
    print("Number of iterations:", i)
    
    #return the answer
    #same identation as while
    return I[i]

## Using these methods to integrate the function

In [ ]:
Answer = fxn_int(np.pi)-fxn_int(0)
print("True answer: ", Answer)
print()

print("Trapezoid")
print("Number of iterations: 3388")
print(trapezoid_method(fxn,0,np.pi,3388))  #from a=0 to b=1 in ten intervals
print()

print("Simpson's method")
print("Number of iterations: 141")
print(simpsons_method(fxn,0,np.pi,141))
print()

print("Romberg Integration")
tolerance = 1.0e-6
RI = romberg_integration(fxn,0,np.pi,tolerance)
print("Final answer:", RI)
print("Error:", (RI-Answer)/Answer)
print("Tolerance:", tolerance)
